# Анализ данных клиентов и их защита

Цель проекта: защитить данные клиентов страховой компании «Хоть потоп». 

Задачи:
- разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию:
- обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

Загружаем данные:

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Для решения задачи можно сразу выделить из исходного датасета целевой признак и остальные признаки объектов:

In [5]:
X = data.drop('Страховые выплаты', axis=1)
X.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


In [6]:
y = data['Страховые выплаты']
y.head()

0    0
1    1
2    0
3    0
4    0
Name: Страховые выплаты, dtype: int64

Данные готовы к дальнейшим преобразованиям.

## 2. Умножение матриц

Для начала обучим модель линейной регрессии и проверим ее качество метрикой RMSE.

In [7]:
model_0 = LinearRegression()
model_0.fit(X, y)
predict_0 = model_0.predict(X)

Посчитаем RMSE модели:

In [8]:
RMSE_0 = np.sqrt(mean_squared_error(y, predict_0))
RMSE_0

0.3512077581960692

Закодируем признаки, умножив их на обратимую матрицу. Приведем формулу:

$$
Xnew = X Z
$$

Матрица Z для перемножения с матрицей X должна иметь длину, равное ширине X. В свою очередь, матрица должна быть обратимой, поэтому ее размерность должна быть равна (4, 4), заполним матрицу значениями от 0 до 10:

In [9]:
Z = np.random.randint(0, 10, (4, 4))
print(Z)

[[3 9 8 6]
 [6 4 3 5]
 [7 2 6 3]
 [8 5 9 7]]


Умножим эту матрицу на матрицу исходных признаков и посчитаем RMSE полученной модели:

In [10]:
X_new = X.values.dot(Z)

In [11]:
X_new

array([[347457.,  99378., 297740., 149018.],
       [266284.,  76189., 228147., 114237.],
       [147174.,  42116., 126087.,  63145.],
       ...,
       [237436.,  67890., 203478., 101814.],
       [229059.,  65512., 196301.,  98237.],
       [284379.,  81326., 243701., 121953.]])

Обучим заново модель линейной регрессии и сравним полученные значения RMSE:

In [12]:
model_1 = LinearRegression()
model_1.fit(X_new, y)
predict_1 = model_1.predict(X_new)

In [13]:
# посчитаем RMSE модели
RMSE_1 = np.sqrt(mean_squared_error(y, predict_1))
RMSE_1

0.35120775819607

**Ответ:** Качество модели не изменилось

**Обоснование:** Посчитаные метрики до и после преобразования говорят о том, что качество моделей одинаково (RMSE_0 = RMSE_1)

Находим оптимальный вектор весов $\omega'$ для преобразованных признаков X_new:

In [18]:
w_new = np.linalg.inv(X_new.T.dot(X_new)).dot(X_new.T).dot(y)
w_new

array([ 0.01413161,  0.01125349, -0.01370458, -0.01307086])

Получается что при найденом $\omega'$ средняя квадратичная ошибка предсказания в задаче линейной регрессии для преобразованных данных будет минимальна.

Вычислим вектор предсказания модели на преобразованных данных:

In [19]:
a_new = X_new.dot(w_new)
a_new

array([ 0.2828728 ,  0.58051362,  0.42948116, ..., -0.02345642,
       -0.05255879,  0.0854748 ])

Но для проверки рассчитаем a:

In [20]:
wX = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
wX

array([-4.43854686e-02,  2.33356224e-02, -1.17739038e-05, -4.55168125e-02])

In [21]:
aX = X.dot(wX).values
aX

array([ 0.28287261,  0.58051347,  0.42948107, ..., -0.02345651,
       -0.05255887,  0.08547465])

Новый вектор предсказаний равен старому:

$a'=a$, следовательно, $X'\omega' = X\omega$

Если продолжить, то

$XZ\omega' = X\omega$

$Z\omega' = \omega$

$\omega' = \omega{(Z^T)^{-1}}$

Проверим правильность вывода

In [22]:
wX @ np.linalg.inv(Z.T)

array([ 0.0141316 ,  0.01125349, -0.01370457, -0.01307085])

Сравним с w_new

In [23]:
w_new

array([ 0.01413161,  0.01125349, -0.01370458, -0.01307086])

**Вывод:**

параметры линейной регрессии в исходной задаче и в преобразованной связаны соотношением $\omega' = \omega{(Z^T)^{-1}}$ , где Z - обратимая матрица с длинной, равной ширине X.

## 3. Алгоритм преобразования

In [24]:
np.linalg.inv(Z)

array([[-0.00683371,  0.16173121,  0.25968109, -0.22095672],
       [ 0.20501139,  0.14806378,  0.2095672 , -0.37129841],
       [ 0.02505695, -0.25968109,  0.04783599,  0.14350797],
       [-0.17084282,  0.04328018, -0.50797267,  0.476082  ]])

Можно создать функуцию, которая будет делать преобразование данных для дальнейшего анализа:

In [25]:
def coding_data(data, target_name):
    # выделяем признаки из данных
    X = data.drop(target_name, axis=1).values
    # выделяем целевой признак
    y = data[target_name]
    # генерируем ключ преобразования
    key = np.random.randint(0, 10, (X.shape[1], X.shape[1]))
    # преобразуем данные
    X_new = X @ key
    # возвращаем преобразованные данные
    return X_new, y, key

Функция каждый раз будет генерировать новый ключ.

Создадим функцию для декодирования признаков (если вдруг это понадобится):

In [26]:
def decoding_data(cod_data, key):
    # умножим закодированную матрицу на обратную ключа и вернем результат
    return cod_data.dot(np.linalg.inv(key))

Итак, алгоритм преобразования и сравнения моделей следующий (первые 4 пункта выполним с помощью функции coding_data):
- 1.1 Извлекаем признаки из исходных данных.
- 1.2 Сохраняем в отдельную переменную целевые признаки объектов.
- 1.3 Генерируем ключ преобразования данных - это квадратная обратимая матрица с размерностью (количество столбцов, количество столбцов) признаков.
- 1.4 Переменожаем признаки на сгенерированную матрицу и получаем преобразованные признаки, целевой признак и ключ.
- 2. Создаем модель линейной регрессии и обучаем ее на непреобразованных данных.
- 3. Считаем метрику R2 (для линейной регрессии можно просто вызвать model.score(X, y))
- 4. Создаем и обучаем модель линейной регрессии на преобразованных данных.
- 5. Считаем метрику R2.
- 6. Сравниваем метрики.

**Обоснование**

При перемножении матрицы признаков на обратимую матрицу получаем следующий результат:


$\begin{pmatrix}
X_{11} & X_{12} & X_{13} & X_{14}\\
X_{21} & X_{22} & X_{23} & X_{24}\\
... & ... & ... & ...\\
X_{n1} & X_{n2} & X_{n3} & X_{n4}\\
\end{pmatrix} \begin{pmatrix}
Z_{11} & Z_{12} & Z_{13} & Z_{14}\\
Z_{21} & Z_{22} & Z_{23} & Z_{24}\\
Z_{31} & Z_{32} & Z_{33} & Z_{34}\\
Z_{41} & Z_{42} & Z_{43} & Z_{44}\\
\end{pmatrix} = 
$

$\begin{pmatrix}
X_{11}Z_{11} + X_{12}Z_{21} + X_{13}Z_{31} + X_{14}Z_{41} & X_{11}Z_{12} + X_{12}Z_{22} + X_{13}Z_{32} + X_{14}Z_{42} & X_{11}Z_{13} + X_{12}Z_{23} + X_{13}Z_{33} + X_{14}Z_{43} & X_{11}Z_{14} + X_{12}Z_{24} + X_{13}Z_{34} + X_{14}Z_{44}\\
X_{21}Z_{11} + X_{22}Z_{21} + X_{23}Z_{31} + X_{24}Z_{41} & X_{21}Z_{12} + X_{22}Z_{22} + X_{23}Z_{32} + X_{24}Z_{42} & X_{21}Z_{13} + X_{22}Z_{23} + X_{23}Z_{33} + X_{24}Z_{43} & X_{21}Z_{14} + X_{22}Z_{24} + X_{23}Z_{34} + X_{24}Z_{44}\\
... & ... & ... & ...\\
X_{n1}Z_{11} + X_{n2}Z_{21} + X_{n3}Z_{31} + X_{n4}Z_{41} & X_{n1}Z_{12} + X_{n2}Z_{22} + X_{n3}Z_{32} + X_{n4}Z_{42} & X_{n1}Z_{13} + X_{n2}Z_{23} + X_{n3}Z_{33} + X_{n4}Z_{43} & X_{n1}Z_{14} + X_{n2}Z_{24} + X_{n3}Z_{34} + X_{n4}Z_{44}\\
\end{pmatrix}$

Обозначим эти суммы через $X^{'}$ и получим преобразованные признаки в виде:

$\begin{pmatrix}
X^{'}_{11} & X^{'}_{12} & X^{'}_{13} & X^{'}_{14}\\
X^{'}_{21} & X^{'}_{22} & X^{'}_{23} & X^{'}_{24}\\
... & ... & ... & ...\\
X^{'}_{n1} & X^{'}_{n2} & X^{'}_{n3} & X^{'}_{n4}\\
\end{pmatrix}$

Для восстановления значений исходных признаков нужно полученную преобразованну матрицу умножить на обратную матрицу-ключ:

$\begin{pmatrix}
X^{'}_{11} & X^{'}_{12} & X^{'}_{13} & X^{'}_{14}\\
X^{'}_{21} & X^{'}_{22} & X^{'}_{23} & X^{'}_{24}\\
... & ... & ... & ...\\
X^{'}_{n1} & X^{'}_{n2} & X^{'}_{n3} & X^{'}_{n4}\\
\end{pmatrix} \begin{pmatrix}
Z_{11} & Z_{12} & Z_{13} & Z_{14}\\
Z_{21} & Z_{22} & Z_{23} & Z_{24}\\
Z_{31} & Z_{32} & Z_{33} & Z_{34}\\
Z_{41} & Z_{42} & Z_{43} & Z_{44}\\
\end{pmatrix}^{-1} = \begin{pmatrix}
X_{11} & X_{12} & X_{13} & X_{14}\\
X_{21} & X_{22} & X_{23} & X_{24}\\
... & ... & ... & ...\\
X_{n1} & X_{n2} & X_{n3} & X_{n4}\\
\end{pmatrix}$

## 4. Проверка алгоритма

Воспользуемся выше описанным алгоритмом.

#### Преобразование исходных данных и получение признаков

In [27]:
X_new, y_new, key = coding_data(data, 'Страховые выплаты')

#### Создаем модель линейной регрессии и обучаем ее на непреобразованных данных.

In [28]:
model_no_coding = LinearRegression().fit(X, y)

#### Считаем метрику R2

In [29]:
R2_no_coding = model_no_coding.score(X, y)
print(f'Качество модели линейной регрессии на непреобразованных данных по метрике R2 равно {R2_no_coding}')

Качество модели линейной регрессии на непреобразованных данных по метрике R2 равно 0.42494550286668


#### Создаем и обучаем модель линейной регрессии на преобразованных данных.

In [30]:
model_coding = LinearRegression().fit(X_new, y_new)

#### Считаем метрику R2

In [31]:
R2_coding = model_coding.score(X_new, y_new)
print(f'Качество модели линейной регрессии на непреобразованных данных по метрике R2 равно {R2_coding}')

Качество модели линейной регрессии на непреобразованных данных по метрике R2 равно 0.42494550286667765


### Вывод

Как видно из расчетов метрик, качество моделей отличается очень незначительно, метрики практически равны. 

Это говорит о том, что разработанный метод преобразования данных позволяет скрыть персональную информацю, не ухудшая при этом качество предсказаний линейной регрессии.

При этом данные пользователей можно легко восстановить по сохраненному ключу.